### https://discourse.julialang.org/t/question-on-the-design-of-macro-hygiene/73992
2022年1月

In [1]:
macro dummy(ex)
    return ex
end

xx = 3
@dummy yy = xx ^ 2 # print 

9

In [4]:
yy # undefvalerror: yy not defined

UndefVarError: UndefVarError: `yy` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

My question is about the design of this feature. In this case, it seems to me intuitive that, given the fact that the user (who calls the dummy macro) has handed the variable name yy to the macro, he should himself be responsible for the naming conflict and stuffs.<br>

的问题是关于这个功能的设计。在这种情况下，我直觉地认为，鉴于用户（调用 dummy 宏）已经将变量名称 yy 交给宏，他应该自己负责命名冲突和东西。

In [5]:
let xx = 3
    @macroexpand @dummy yy = xx ^ 2
end

:(var"#64#yy" = Main.xx ^ 2)

+ as you observed, `yy` has been gensymmed by the compiler; this is because it was considered a local variable in the context of the macro<br>
如您所见， `yy` 已被编译器写入 `gensym` ；这是因为它被认为是宏上下文中的局部变量<br>

+ maybe more subtly, `xx` refers to `Main.xx` even though the `xx` variable is scoped in the `let` block in the context of the macro call. This is because this variable was considered a global variable in the context of the macro expansion.<br>
也许更微妙的是， `xx` 指的是 `Main.xx` ，即使在宏调用的上下文中 `xx` 变量的作用域在 `let` 块中。这是因为该变量在宏观展开的上下文中被视为全局变量。

If you, as a macro developer, want to disable automatic hygiene on some part of the generated expression, you can do so using the `esc` function (and in many cases you actually should disable automatic hygiene handling for user-provided expressions).<br>

我的理解是，在Julia中做出的选择可能是谨慎的错误，通过使用这些规则自动处理卫生问题。作为一个宏开发人员，如果你想在生成的表达式的某些部分禁用自动卫生，你可以使用 `esc` 函数（在许多情况下，你实际上应该禁用用户提供的表达式的自动卫生处理）。

In [6]:
macro dummy2(ex)
    return esc(ex)
end

@dummy2 (macro with 1 method)

In [9]:
let x = 3
    @dummy2 y = x ^ 2

    @show y
    @macroexpand @dummy2 y = x ^ 2
end

y = 9


:(y = x ^ 2)

On a personal note, having spent a fair amount of time developing macros in (Emacs)LISP where you have to manually take care of gensymming everything, I can say that always having to think about hygiene puts an extra burden on the macro developer. So I kind of like Julia’s approach (although it did take me some time to get used to it).<br>

就我个人而言，我花了相当多的时间在（Emacs）LISP中开发宏，你必须手动处理所有的gensymming，我可以说，总是不得不考虑卫生给宏开发人员带来了额外的负担。所以我有点喜欢Julia的方法（尽管我花了一些时间来适应它）。

Not sure I know all the details here, but my understanding is that it can be extremely difficult to make the distinction between what actually comes from the user-provided expression and the macro itself.<br>
我不确定我了解这里的所有细节，但我的理解是，区分来自用户提供的表达式和宏本身的实际内容是非常困难的。<br>

That distinction might not even make so much sense for more complex macros. Take for example the macros from `Base.Cartesian`:<br>
对于更复杂的宏，这种区别甚至可能没有太大意义。以 `Base.Cartesian` 的宏为例：

In [10]:
@macroexpand Base.Cartesian.@nexprs 4 i -> x_i = i

quote
    x_1 = 1
    x_2 = 2
    x_3 = 3
    x_4 = 4
end

None of the `x_1`, `x_2`, … symbols actually come from the context of the macro call; they were all built by the macro itself, and are going to be injected as new variable names in the context of the macro call. In such a case, you can’t really “trace back” what `x_i` refers to in the macro call.<br>

 `x_1` ,  `x_2` ，…这些符号实际上都不是来自宏调用的上下文；它们都是由宏本身构建的，并且将在宏调用的上下文中作为新的变量名注入。在这种情况下，您无法真正“追溯”宏调用中 `x_i` 所指的内容。<br>

 Julia’s take is to be very flexible. It handles hygiene in a way that can be automated and is sensible in a lot of common cases. And it also allows you to handle everything manually by:<br>
茱莉亚的做法是非常灵活。它以一种可以自动化的方式处理卫生问题，并且在许多常见情况下是明智的。它还允许您手动处理所有内容：<br>

+ **using `esc` to disable auto hygiene,** and<br>
使用 `esc` 禁用自动卫生<br>

+ **using `gensym` explicitly to inject symbols that won’t clash in the macro output.**<br>
显式地使用 `gensym` 注入不会在宏输出中冲突的符号。

reply by another peopele:<br>

Early on (2014), there was a proposal to do just this — basically, only symbols defined inside the macro should be hygienized, whereas symbols coming from macro arguments should be left alone, eliminating the need for esc: https://github.com/JuliaLang/julia/pull/6910<br>

早些时候（2014年），有人建议这样做——基本上，只有宏内部定义的符号应该被卫生，而来自宏参数的符号应该被单独保留，从而消除了 esc : https://github.com/JuliaLang/julia/pull/6910的需要<br>

The Julia developers were pretty receptive to the idea, but the PR ran into some technical problems and languished for lack of attention. It’s the sort of thing that might still be considered for Julia 2.0.<br>

**Julia的开发人员非常乐于接受这个想法，但是PR遇到了一些技术问题，并且由于缺乏关注而萎靡不振。这是Julia 2.0可能会考虑的事情。**